In [1]:
schema = 'acurtis'
player = 2535445291321133

In [2]:
import json
import pathlib

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

from halo_infinite_analysis import db

In [3]:
ROOT = pathlib.Path('C:/Users/acurt/source/halo-infinite-analysis')
CONFIG = json.loads((ROOT / 'config.json').read_text())

In [100]:
sql = """
    SELECT *
    FROM "acurtis".vw_player
    WHERE
        xuid = 2535445291321133 AND
        playlist_experience = 'Arena'
"""

with db.connect(**CONFIG['database']) as conn:
    df = pd.read_sql(sql, conn)

df['match_start_time'] = pd.to_datetime(df['match_start_time'], utc=True)
df['match_start_time'] = df['match_start_time'].dt.tz_convert('US/Central')
df.head()

C:\Users\acurt\AppData\Local\Temp\ipykernel_2608\1747109646.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,id,match_id,player_id,xuid,match_start_time,match_duration,lifecycle_mode,game_variant_category,map_variant,game_variant,...,driver_assists,emp_assists,vehicle_destroys,hijacks,max_killing_spree,pre_match_csr,post_match_csr,kills_expected,deaths_expected,team_mmr
0,1668,1,4419,2535445291321133,2022-03-11 23:16:36.135000-06:00,714.87750,Matchmade,CTF,Bazaar,Arena:CTF,...,0,0,0,0,6,0,0,21.233072,10.857877,1210.6848
1,8349,2,4419,2535445291321133,2022-02-04 22:46:42.924000-06:00,741.22614,Matchmade,CTF,Aquarius,Arena:CTF,...,0,0,0,0,9,0,0,20.693083,11.117517,1084.9893
2,9501,4,4419,2535445291321133,2022-04-05 20:26:39.084000-05:00,867.21410,Matchmade,Oddball,Live Fire,Arena:Oddball,...,0,0,0,0,6,0,0,25.539206,20.475739,1270.4967
3,7182,5,4419,2535445291321133,2022-07-05 23:12:02.899000-05:00,815.04230,Matchmade,Bastion,Streets,Ranked:King of the Hill,...,0,0,0,0,4,1527,1529,27.724669,17.582087,1536.9445
4,1049,6,4419,2535445291321133,2022-09-27 21:01:21.077000-05:00,551.05664,Matchmade,Fiesta,Catalyst,Fiesta:Slayer,...,0,0,0,0,7,0,0,15.522384,11.035573,1260.9906


In [14]:
df['outcome'].value_counts(True)

Win             0.544434
Loss            0.422825
Tie             0.016838
DidNotFinish    0.015903
Name: outcome, dtype: float64

In [15]:
def outcome_percent_by(key):
    return (
        df.groupby(key)
            .filter(lambda x: len(x) > 10)
            .groupby(key)
            ['outcome']
            .value_counts(True)
            .unstack(len(key) if isinstance(key, list) else 1)
            .style
            .format(formatter='{:.1%}', na_rep='-')
            .background_gradient()
    )

In [16]:
outcome_percent_by('game_variant_category')

outcome,DidNotFinish,Loss,Tie,Win
game_variant_category,,,,
Attrition,-,54.5%,3.6%,41.8%
Bastion,3.3%,37.8%,1.1%,57.8%
CTF,0.5%,38.5%,7.7%,53.3%
Fiesta,1.3%,35.5%,-,63.2%
Oddball,2.7%,48.2%,-,49.1%
Slayer,1.9%,40.2%,0.2%,57.7%
Strongholds,0.9%,52.6%,-,46.6%


In [17]:
outcome_percent_by('playlist')

outcome,DidNotFinish,Loss,Tie,Win
playlist,,,,
Fiesta,1.3%,36.0%,-,62.7%
Quick Play,1.2%,43.2%,2.7%,52.9%
Ranked Arena,3.1%,46.0%,-,50.9%
Ranked Doubles,-,61.9%,-,38.1%
Social Slayer,-,33.3%,13.3%,53.3%
Tactical Slayer,5.9%,17.6%,-,76.5%
Team Doubles,-,45.2%,3.2%,51.6%
Team Slayer,-,37.2%,-,62.8%
Team Snipers,1.4%,34.3%,-,64.3%


In [18]:
outcome_percent_by('map_variant')

outcome,DidNotFinish,Loss,Tie,Win
map_variant,,,,
Aquarius,0.8%,39.5%,0.8%,58.9%
Bazaar,1.0%,41.3%,2.9%,54.8%
Behemoth,1.0%,52.5%,1.0%,45.5%
Catalyst,-,41.0%,-,59.0%
Launch Site,5.4%,27.2%,10.9%,56.5%
Live Fire,2.2%,40.8%,0.6%,56.4%
Recharge,0.6%,46.9%,0.6%,52.0%
Streets,2.2%,45.7%,0.5%,51.6%


In [19]:
outcome_percent_by(['map_variant', 'game_variant_category'])

In [101]:
outcome_percent_by(df['match_start_time'].dt.hour)

outcome,DidNotFinish,Loss,Tie,Win
match_start_time,,,,
0,-,42.9%,-,57.1%
12,-,15.4%,-,84.6%
13,-,33.3%,-,66.7%
14,-,46.7%,-,53.3%
15,-,47.6%,4.8%,47.6%
16,9.1%,63.6%,-,27.3%
20,1.2%,45.1%,3.0%,50.6%
21,1.7%,37.0%,1.4%,59.9%
22,1.2%,43.4%,1.4%,54.0%


In [102]:
df['match_start_time'].dt.hour.value_counts().sort_index()

0      14
12     13
13     18
14     15
15     21
16     11
19      6
20    164
21    292
22    346
23    169
Name: match_start_time, dtype: int64

In [104]:
outcome_percent_by(df['match_start_time'].dt.dayofweek)

outcome,DidNotFinish,Loss,Tie,Win
match_start_time,,,,
0,-,46.7%,13.3%,40.0%
1,2.0%,41.2%,2.0%,54.8%
2,-,39.3%,7.1%,53.6%
3,1.1%,46.7%,-,52.2%
4,1.7%,40.4%,-,57.9%
5,-,54.2%,1.2%,44.6%
6,-,29.6%,-,70.4%


In [105]:
df['match_start_time'].dt.dayofweek.value_counts().sort_index()

0     15
1    646
2     28
3     92
4    178
5     83
6     27
Name: match_start_time, dtype: int64

In [88]:
def metric_by(metric, key, agg=np.mean, time_normalized=False):
    _df = df[:]
    if time_normalized:
        for m in metric:
            _df[m] /= _df['time_played']
            _df[m] *= _df['time_played'].mean()
    grouped = (
        _df.groupby(key)
            .filter(lambda x: len(x) > 10)
            .groupby(key)
    )
    out = grouped[metric].agg(agg)
    
    if isinstance(out, pd.Series):
        out = out.to_frame()
    return (
        out.style
            .format(formatter='{:.1f}', na_rep='-')
            .background_gradient()
    )

In [89]:
metric_by(['kills', 'deaths', 'assists', 'kda'], 'game_variant_category', time_normalized=True)

,kills,deaths,assists,kda
game_variant_category,,,,
Attrition,12.6,8.7,4.8,4.8
Bastion,17.4,11.7,8.4,8.2
CTF,14.6,8.7,4.9,7.2
Fiesta,15.0,10.4,4.1,5.6
Oddball,17.4,10.7,7.0,8.7
Slayer,15.5,9.8,5.3,7.2
Strongholds,15.5,10.4,5.3,6.4


In [92]:
metric_by(['kills', 'deaths', 'assists', 'kda'], 'playlist', time_normalized=True)

,kills,deaths,assists,kda
playlist,,,,
Fiesta,15.0,10.4,4.0,5.6
Quick Play,15.6,9.5,6.1,7.8
Ranked Arena,15.3,11.1,6.3,6.0
Ranked Doubles,12.4,8.5,2.5,4.3
Social Slayer,15.6,11.7,5.1,5.3
Tactical Slayer,21.6,12.8,1.1,8.9
Team Doubles,13.9,9.0,3.4,5.6
Team Slayer,14.6,9.5,6.3,6.9
Team Snipers,16.5,9.8,3.0,7.4


In [94]:
metric_by(['kills', 'deaths', 'assists', 'kda'], 'map_variant', time_normalized=False)

,kills,deaths,assists,kda
map_variant,,,,
Aquarius,14.6,9.5,5.6,6.5
Bazaar,17.2,11.0,6.0,8.0
Behemoth,15.6,9.4,4.6,7.3
Catalyst,14.2,8.4,4.5,6.9
Launch Site,13.9,8.7,4.0,6.2
Live Fire,16.2,10.5,6.2,7.5
Recharge,15.4,10.1,5.5,6.7
Streets,16.4,11.0,6.8,7.3


In [96]:
metric_by(['kills', 'deaths', 'assists', 'kda'], ['map_variant', 'game_variant_category'], time_normalized=True)